In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D, ReLU, Flatten

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, mean_squared_error, mean_absolute_error

In [ ]:
# no_of_noise_samples = 5000
no_of_merged_samples = 5000
no_of_params = 2
directory = "/content/gdrive/MyDrive/GW_SOP/Code/debugging/1D-CNN BBH Parameter Estimation/"+str(no_of_params)+"_parameters/Signal_HIDDEN_in_strain_data/White_noise/Test 5/data/"

In [ ]:
# noise_df = pd.read_csv(directory + "Final_Merged_Noise_Reduced_No_Abs.csv", header=None)
# noise = noise_df.values.astype(float)

# noise_targets = np.zeros((no_of_noise_samples, no_of_params))


data_BBH_df = pd.read_csv(directory + "Final_BBH_Merged_Noise_Signal.csv", header=None)
data_BBH = data_BBH_df.values.astype(float)

data_BBH_df_targets = pd.read_csv(directory + "Final_BBH_Merged_Noise_Signal_Targets.csv", header=None)
data_BBH_targets = data_BBH_df_targets.values.astype(float)

In [ ]:
# X = np.concatenate((noise, data_BBH), axis=0)
X = data_BBH
print(X.shape)

In [ ]:
# y = np.concatenate((noise_targets, data_BBH_targets), axis=0)
y = data_BBH_targets

print(y)
print(y.shape)

In [ ]:
X *= 1e19
print(X)

In [ ]:
X = np.expand_dims(X, axis=-1)
print(X.shape)

In [ ]:
X_train, X1, y_train, y1 = train_test_split(X, y, test_size=0.2)

X_val, X_test, y_val, y_test = train_test_split(X1, y1, test_size=0.5)

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
model = Sequential()

model.add(Conv1D(16, 16, input_shape = (16384,1)))
model.add(MaxPool1D(4, 4))
model.add(ReLU())
model.add(Conv1D(32, 8))
model.add(MaxPool1D(4, 4))
model.add(ReLU())
model.add(Conv1D(64, 8))
model.add(MaxPool1D(4, 4))
model.add(ReLU())
model.add(Conv1D(128, 8))
model.add(MaxPool1D(4, 4))
model.add(ReLU())
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation=None))

model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='mean_squared_error')

In [ ]:
history = model.fit(X_train, y_train, batch_size=128, epochs=5, validation_data = (X_val, y_val))

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = [i for i in range(len(loss))]
plt.plot(epochs, loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.xticks(np.arange(min(epochs), max(epochs)+1, 1))
plt.ylabel('Mean Squared Error')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.savefig(directory[:-5]+"screenshots/graph_1.png")
plt.show()

In [ ]:
# Training Data

pred = model.predict(X_train)
y_pred = pred
y_true = y_train

mse = mean_squared_error(y_true, y_pred)
print('Mean Squared Error: %.3f' % mse)

mae = mean_absolute_error(y_true, y_pred)
print('Mean Absolute Error: %.3f' % mae)

In [ ]:
# Validation Data

pred = model.predict(X_val)
y_pred = pred
y_true = y_val

mse = mean_squared_error(y_true, y_pred)
print('Mean Squared Error: %.3f' % mse)

mae = mean_absolute_error(y_true, y_pred)
print('Mean Absolute Error: %.3f' % mae)

In [ ]:
# Testing Data

pred = model.predict(X_test)
y_pred = pred
y_true = y_test

mse = mean_squared_error(y_true, y_pred)
print('Mean Squared Error: %.3f' % mse)

mae = mean_absolute_error(y_true, y_pred)
print('Mean Absolute Error: %.3f' % mae)

In [ ]:
# model.save(directory + "model_trial_1.h5")